# Advanced Intent Recognition and Entity Extraction

This notebook demonstrates production-grade techniques for NLU using modern machine learning approaches.

In [1]:
# Install required libraries
!pip install transformers datasets scikit-learn nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 26.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 29.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 58.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [6]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForTokenClassification
from transformers import pipeline
import torch
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [5]:
import tqdm as notebook_tqdm

## 1. Intent Recognition with Transformer Models

Production systems use pre-trained language models fine-tuned on domain-specific data.

In [7]:
# Create sample intent data
intent_examples = {
    'booking': [
        "I want to book a flight to New York",
        "Can you reserve a table for two on Friday?",
        "I need to schedule an appointment",
        "Book me a taxi for tomorrow morning"
    ],
    'weather': [
        "What's the weather like in London?",
        "Is it going to rain tomorrow?",
        "Tell me the forecast for this weekend",
        "How hot will it be today?"
    ],
    'order_food': [
        "I want to order a pizza",
        "Can I get a large pepperoni with extra cheese?",
        "Order me some takeout",
        "I'd like to order dinner"
    ]
}

# Convert to DataFrame format for training
texts = []
labels = []

for intent, examples in intent_examples.items():
    for example in examples:
        texts.append(example)
        labels.append(intent)

intent_df = pd.DataFrame({'text': texts, 'intent': labels})
intent_df.sample(10)

,text,intent
8,I want to order a pizza,order_food
11,I'd like to order dinner,order_food
6,Tell me the forecast for this weekend,weather
1,Can you reserve a table for two on Friday?,booking
7,How hot will it be today?,weather
0,I want to book a flight to New York,booking
5,Is it going to rain tomorrow?,weather
10,Order me some takeout,order_food
3,Book me a taxi for tomorrow morning,booking
9,Can I get a large pepperoni with extra cheese?,order_food


### Using Pre-trained Transformer Model for Intent Classification

In production, we would fine-tune a model on thousands of examples, but we'll use a pre-trained model to demonstrate.

In [ ]:
# Load a pre-trained zero-shot classification model
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Function to classify intents using the model
def classify_intent_transformer(text, candidate_labels):
    result = classifier(text, candidate_labels)
    return {
        "intent": result['labels'][0],
        "confidence": result['scores'][0],
        "all_intents": dict(zip(result['labels'], result['scores']))
    }

# Example
sample_text = "I want to reserve a table for two at an Italian restaurant"
intent_result = classify_intent_transformer(sample_text, list(intent_examples.keys()))
print(f"Text: {sample_text}")
print(f"Detected Intent: {intent_result['intent']} (confidence: {intent_result['confidence']:.4f})")
print("All intents with confidence scores:")
for intent, score in intent_result['all_intents'].items():
    print(f"  - {intent}: {score:.4f}")

## 2. Named Entity Recognition with Transformer Models

Production systems use specialized token classification models for entity extraction.

In [ ]:
# Load a pre-trained NER model
ner_pipeline = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")

def extract_entities_transformer(text):
    entities = ner_pipeline(text)
    
    # Format the results
    formatted_entities = {}
    for entity in entities:
        entity_type = entity['entity_group']
        if entity_type in formatted_entities:
            if isinstance(formatted_entities[entity_type], list):
                formatted_entities[entity_type].append(entity['word'])
            else:
                formatted_entities[entity_type] = [formatted_entities[entity_type], entity['word']]
        else:
            formatted_entities[entity_type] = entity['word']
    
    return formatted_entities

# Example
entity_texts = [
    "I want to fly from New York to San Francisco next Friday",
    "John Smith works at Microsoft in Seattle",
    "I need to meet Sarah Johnson at the coffee shop on Main Street"
]

for text in entity_texts:
    entities = extract_entities_transformer(text)
    print(f"Text: {text}")
    print(f"Extracted Entities: {entities}")
    print("-" * 80)

## 3. Combined Intent and Entity Processing

In production systems, intent recognition and entity extraction work together in a pipeline.

In [ ]:
def analyze_text_production(text, intent_labels):
    # Step 1: Intent Recognition
    intent_result = classify_intent_transformer(text, intent_labels)
    
    # Step 2: Entity Extraction
    entities = extract_entities_transformer(text)
    
    # Step 3: Combine Results
    return {
        "text": text,
        "intent": intent_result["intent"],
        "intent_confidence": intent_result["confidence"],
        "entities": entities
    }

# Testing with complex examples
test_examples = [
    "Book me a flight from London to Paris for next Tuesday",
    "What's the weather going to be like in New York this weekend?",
    "I'd like to order a large pepperoni pizza with extra cheese from Domino's"
]

for example in test_examples:
    result = analyze_text_production(example, list(intent_examples.keys()))
    print(f"Text: {result['text']}")
    print(f"Intent: {result['intent']} (confidence: {result['intent_confidence']:.4f})")
    print(f"Entities: {result['entities']}")
    print("-" * 80)

## 4. Fine-tuning Process for Production Systems

In real-world applications, these models would be fine-tuned on domain-specific data:

In [ ]:
# Pseudo-code for fine-tuning process (not meant to be executed)
"""
# 1. Data collection and annotation
- Collect thousands of examples specific to your domain
- Have human annotators label intents and entities
- Split into train/validation/test sets

# 2. Intent classifier fine-tuning
from transformers import Trainer, TrainingArguments
from datasets import Dataset

# Convert data to HuggingFace datasets
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

# Load pre-trained model
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(intents))
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Fine-tune
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

trainer.train()

# 3. NER model fine-tuning (similar process)

# 4. Deploy models to production
- Export to ONNX or TorchScript for faster inference
- Set up monitoring for model performance
- Implement continuous learning and model updating
"""

## 5. Advanced Techniques Used in Production

1. **Few-shot and zero-shot learning** - Handling new intents without extensive training
2. **Confidence thresholds** - Falling back to default behaviors when uncertain
3. **Multi-intent detection** - Recognizing multiple intents in a single utterance
4. **Context tracking** - Maintaining conversation state across multiple turns
5. **Hybrid approaches** - Combining rule-based systems with ML for high reliability
6. **Active learning** - Continuously improving models from user interactions